In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score,cross_val_predict
from sklearn.metrics import roc_curve, auc
import matplotlib
matplotlib.rcParams.update({'font.size': 20})
import pickle
import platform
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from numpy import interp
from sklearn.utils.multiclass import type_of_target
import os

In [ ]:
def read_csv(csv_file, nrows=None):
    df = pd.read_csv(csv_file, nrows=nrows)
    print("File = {}".format(csv_file))
    print("Shape = {:,} rows, {:,} columns".format(df.shape[0], df.shape[1]))
    print("Memory usage = {:.2f}GB".format(df.memory_usage().sum() / 1024**3))
    return df

data_dir = ".../data/"

df = read_csv(data_dir + "NEN_ml_5k_res_test_update.csv")

In [ ]:
fdf = df.iloc[:,list(range(2, len(df.columns),1))]

fdf = fdf.drop(labels = ['Age.at.diagnosis', "Sentrix_ID_new", 'Ki67', 'index','Localization','Primary','G.phase', 'Gender'],axis = 1)
fdf_cup = fdf.loc[fdf['P_grouping'] == "NEN liver CUP",:]
fdf_cup = fdf_cup.drop(fdf_cup[fdf_cup['ID'] == 240230].index)
fdf_meta = fdf.loc[fdf['P_grouping'] == "NEN liver metastasis",:]
fdf_model = fdf.drop(fdf[(fdf['P_grouping'] == "NEN liver metastasis") | (fdf['P_grouping'] == "NEN liver CUP" )].index)
list_data = [fdf_cup, fdf_meta, fdf_model]
# Label the class
for i,dataset in enumerate(list_data):
    le = LabelEncoder()
    le_count = 0
    ref_encoding = []

    # iterate through columns
    for col in dataset:
        if dataset.loc[:, col].dtype == 'object':
            if len(list(dataset.loc[:, col].unique())) <= 20:
                le.fit(dataset.loc[:, col])
                dataset.loc[:, col] = le.transform(dataset.loc[:, col])
                le_count += 1
                ref_encoding.append(le.classes_)

    print('%d columns were label encoded.' % le_count)

In [ ]:
fdf_model['P_grouping'].value_counts()
fdf_model['P_grouping'].head(4)

fdf_model['P_grouping'].plot.hist()
plt.show()

In [ ]:
#%% model setting

y = fdf_model.P_grouping
df_pg = fdf_model.drop(['ID','NEN.type','P_grouping'],axis = 1)     #### Keep only the predictive features
X = df_pg.copy()

print("Shape of input data: {} and shape of target variable: {}".format(X.shape, y.shape))
pd.concat([X, y], axis=1).head()

In [ ]:
df_pg = fdf_meta.drop(['ID','NEN.type','P_grouping'],axis = 1)
y_test = fdf_meta.P_grouping   
X_test = df_pg.copy()

df_pg_cup = fdf_cup.drop(['ID','NEN.type','P_grouping'],axis = 1)
X_test_cup = df_pg_cup.copy()


print("Shape of input data: {} and shape of target variable: {}".format(X_test.shape, y_test.shape))
pd.concat([X_test, y_test], axis=1).head()

In [ ]:
y = pd.to_numeric(y)
# The folds are made by preserving the percentage of samples for each class.
kf = StratifiedKFold(n_splits=3)
counter_kf = 1
for train_index, test_index in kf.split(X, y):
    print(f'Fold:{counter_kf}, Train set: {len(train_index)}, Test set:{len(test_index)}')
    counter_kf+=1

In [ ]:
score = cross_val_score(RandomForestClassifier(n_estimators=2000, random_state = 3,max_features="sqrt",
                                               criterion="gini", oob_score=True,
                                                n_jobs=10, max_depth=12),
                                               X, y, cv= kf, scoring="accuracy")
print(f'Scores for each fold are: {score}')
print(f'Average score: {"{:.2f}".format(score.mean())}')

In [ ]:
# Print out the CV score
y_pred = cross_val_predict(RandomForestClassifier(n_estimators=2000, random_state = 3,max_features="sqrt",
                                               criterion="gini", oob_score=True,
                                                n_jobs=10, max_depth=12),
                                               X, y, cv= kf)
conf_mat = confusion_matrix(y,y_pred)

confusion_matrix_cv = pd.DataFrame(conf_mat, columns = ref_encoding[1], index = ref_encoding[1])
#confusion_matrix_cv.to_csv(data_dir + 'RF_results/confusion_matrix2_val.csv', sep=',')

In [ ]:
#%%#%% RF model

random_forest = RandomForestClassifier(n_estimators=1500, random_state=3, max_features="sqrt",
                                       criterion="gini", oob_score=True, n_jobs=10, max_depth=12,
                                       verbose=0)

random_forest.fit(X, y)
print(random_forest.oob_score_)

In [ ]:
testp = random_forest.predict(X_test)
testproba = random_forest.predict_proba(X_test)

testproba_columns = ref_encoding[1].tolist()
testproba= pd.DataFrame(testproba,columns = testproba_columns)

In [ ]:
testproba

In [ ]:
# map it back to the origin meta matrix anno

testprrediction =[ref_encoding[1][i] for i in testp]
testproba_df = pd.DataFrame(testproba)

fdf_1 = df.iloc[:,list(range(2, len(df.columns),1))]
fdf_meta_result = fdf_1.loc[fdf_1['P_grouping'] == "NEN liver metastasis",:]
fdf_meta_result['prediction'] = testprrediction
fdf_comp = fdf_meta_result.loc[:, ('ID','prediction','Primary')]

fdf_comp.reset_index(drop=True, inplace=True)
testproba_df.reset_index(drop=True, inplace=True)

fdf_comp_met = pd.concat([fdf_comp,testproba_df], axis=1, ignore_index=True)
fdf_comp_met.columns = ['ID','prediction','Primary']+testproba_columns
#testproba.to_csv(data_dir +'res_meta_predictions probability.csv', sep=',')
fdf_comp_met.to_csv(data_dir+'RF_results/val_lmc3_results/res_meta_predictions probability.csv', sep=',')

In [ ]:
fdf_comp_met

In [ ]:
testp_cup = random_forest.predict(X_test_cup)
testproba_cup = random_forest.predict_proba(X_test_cup)

testproba_columns = ref_encoding[1].tolist()
testproba_cup= pd.DataFrame(testproba_cup,columns = testproba_columns)

In [ ]:
# map it back to the origin meta matrix anno

testprrediction_cup =[ref_encoding[1][i] for i in testp_cup]
testproba_cup_df = pd.DataFrame(testproba_cup)

fdf_1 = df.iloc[:,list(range(2, len(df.columns),1))]
fdf_cup_result = fdf_1.loc[fdf_1['P_grouping'] == "NEN liver CUP",:]
fdf_cup_result = fdf_cup_result.drop(fdf_cup_result[fdf_cup_result['ID'] == 240230].index)
fdf_cup_result['prediction'] = testprrediction_cup
fdf_comp_cup = fdf_cup_result.loc[:, ('ID','prediction','Primary')]

fdf_comp_cup.reset_index(drop=True, inplace=True)
testproba_cup_df.reset_index(drop=True, inplace=True)

fdf_comp_cup2 = pd.concat([fdf_comp_cup,testproba_cup_df], axis=1, ignore_index=True)
fdf_comp_cup2.columns = ['ID','prediction','Primary']+testproba_columns
fdf_comp_cup2.to_csv(data_dir +'RF_results/val_lmc3_results/res_cup_predictions probability.csv', sep=',')

In [ ]:
fdf_comp_cup2

In [ ]:
#%% feature importance

feature_importance_values = random_forest.feature_importances_
feature_importances = pd.DataFrame({'feature': X.columns, 'importance': feature_importance_values})
# feature_importances.to_csv(os.getcwd() + '/featureimp_res_5k.txt', sep='\t')
#%%
# %% CV_AUC
X = X.values
y = y.values

In [ ]:
from sklearn.metrics import roc_curve, auc
import numpy as np
import matplotlib.pyplot as plt

P_group = ref_encoding[1]
conl = list()
indexl = list()
for i in P_group:
    conl.append(i)
    index = np.where(ref_encoding[1] == i)[0]
    indexl.append(index[0])

for idx, item in zip(indexl, conl):
    print(idx, item)
    classifier = random_forest

    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    plt.figure(figsize=(14, 9))
    i = 0
    for train, test in kf.split(X, y):
        probas_ = classifier.fit(X[train], y[train]).predict_proba(X[test])
        fpr, tpr, thresholds = roc_curve(y[test], probas_[:, idx], pos_label=idx)
        tprs.append(np.interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)  # Ensure auc is not redefined elsewhere
        aucs.append(roc_auc)
        plt.plot(fpr, tpr, lw=1, alpha=0.3,
                 label=f'ROC fold {i} (AUC = {roc_auc:.2f})')
        i += 1

    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
             label='Chance', alpha=.8)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    plt.plot(mean_fpr, mean_tpr, color='b',
             label=f'Mean ROC (AUC = {mean_auc:.2f} ± {std_auc:.2f})',
             lw=2, alpha=.8)

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                     label=r'± 1 std. dev.')

    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'{item}_Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    #plt.savefig(f'{data_dir}RF_results/val_lmc3_results/{item}_ROC_AUC_3CV.svg', format="svg")
    #plt.savefig(f'{data_dir}RF_results/val_lmc3_results/{item}_ROC_AUC_3CV.png')

In [ ]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    roc_auc_score,
    classification_report
)

# Evaluate model performance with additional metrics
# Use cross-validation predictions for evaluation
y_pred = cross_val_predict(
    RandomForestClassifier(n_estimators=2000, random_state=3, max_features="sqrt",
                           criterion="gini", oob_score=True, n_jobs=10, max_depth=12),
    X, y, cv=kf
)

# Confusion Matrix
conf_mat = confusion_matrix(y, y_pred)
print("Confusion Matrix:\n", conf_mat)

# Calculate accuracy
accuracy = accuracy_score(y, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Calculate precision, recall, and F1-score
precision = precision_score(y, y_pred, average="weighted")  # Use 'macro' or 'micro' as needed
recall = recall_score(y, y_pred, average="weighted")
f1 = f1_score(y, y_pred, average="weighted")

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# AUC-ROC score (if applicable)
try:
    auc = roc_auc_score(pd.get_dummies(y), pd.get_dummies(y_pred), average="weighted", multi_class="ovr")
    print(f"AUC-ROC Score: {auc:.2f}")
except ValueError as e:
    print("AUC-ROC Score could not be calculated:", e)

# Classification report
print("\nClassification Report:\n", classification_report(y, y_pred))

# Save metrics to a CSV file
metrics = {
    "Metric": ["Accuracy", "Precision", "Recall", "F1-Score", "AUC-ROC"],
    "Score": [accuracy, precision, recall, f1, auc if 'auc' in locals() else None]
}

metrics_df = pd.DataFrame(metrics)
# metrics_df.to_csv('/mnt/std-pool/homedirs/dtabbakh/nen_project/medcom_2/ML_nen/model_metrics.csv', index=False)

# Save confusion matrix
confusion_matrix_df = pd.DataFrame(conf_mat, columns=ref_encoding[1], index=ref_encoding[1])
# confusion_matrix_df.to_csv('/mnt/std-pool/homedirs/dtabbakh/nen_project/medcom_2/ML_nen/confusion_matrix.csv')

# Plot confusion matrix for better visualization
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(confusion_matrix_df, annot=True, fmt="d", cmap="Blues", xticklabels=ref_encoding[1], yticklabels=ref_encoding[1])
plt.title("Confusion Matrix")
plt.ylabel("Actual")
plt.xlabel("Predicted")
# plt.savefig('.../data/confusion_matrix.png')
plt.show()

In [ ]:
# --- assume you still have ---
# df         : your original DataFrame (with the 'ID' and 'Primary' columns)
# fdf_model  : the subset you trained on (before dropping ID)
# X, y       : your feature matrix & true labels as numpy arrays
# kf         : your StratifiedKFold instance

from sklearn.model_selection import cross_val_predict

# 1) get the predictions again
y_pred = cross_val_predict(
    RandomForestClassifier(
        n_estimators=2000,
        random_state=3,
        max_features="sqrt",
        criterion="gini",
        oob_score=True,
        n_jobs=10,
        max_depth=12
    ),
    X, y, cv=kf
)

# 2) pull out the IDs from fdf_model
ids = fdf_model['ID'].values

# 3) build a small results DataFrame
results = pd.DataFrame({
    'ID':        ids,
    'True':      y,
    'Predicted': y_pred
})

# 4) filter to the misclassified
mis = results[results['True'] != results['Predicted']]

# 5) merge in the 'Primary' label from your original df
mis = mis.merge(df[['ID','Primary']], on='ID', how='left')

# 6) show just the three columns you asked for
print(f"Found {len(mis)} misclassified samples:\n")

target_labels = list(ref_encoding[1])

# 2) build a mapping from code → label
label_map = { i: lbl for i, lbl in enumerate(target_labels) }

# 3) overwrite the numeric codes in 'Predicted' with the string names
mis['Predicted'] = mis['Predicted'].map(label_map)

# 4) show just the three columns you asked for
print(mis[['ID','Primary','Predicted']])

# (optional) save to CSV
mis[['ID','Primary','Predicted']].to_csv(
    data_dir + 'RF_results/misclassified_table.csv',
    index=False
)


In [ ]:
unseen_data = read_csv(".../data/test_output/NEN_ml_5k_res_val_update.csv")

In [ ]:
unseen_data

In [ ]:
# 2. Prepare the features for prediction 
# In training, you dropped ['ID', 'NEN.type', 'P_grouping'] to form X.
X_test_new = unseen_data.drop(['ID', 'NEN.type', 'P_grouping', 'Localization', 'Primary','index'], axis=1)
#X_test_new = unseen_data.drop(['ID', 'NEN.type', 'P_grouping', 'Localization', 'Primary'], axis=1)

In [ ]:
X_test_new

In [ ]:
# 3. Use the trained model to make predictions 
# Predict the class labels (as numbers) and the class probabilities
preds_numeric = random_forest.predict(X_test_new)
preds_proba = random_forest.predict_proba(X_test_new)

In [ ]:
# 4. Convert numeric predictions back to original class names 
# Here we assume that ref_encoding[3] holds the original classes (e.g., ['NEN liver metastasis', 'NEN liver CUP', ...])
target_labels = ref_encoding[1].tolist()  # adjust the index if needed
predicted_labels = [target_labels[i] for i in preds_numeric]

In [ ]:
# 5. Build the final results DataFrame
# Create a DataFrame for the probabilities with appropriate column names
preds_proba_df = pd.DataFrame(preds_proba, columns=target_labels)

In [ ]:
preds_proba_df

In [ ]:
results_df = unseen_data[['ID', 'Primary']].copy()
results_df['prediction'] = predicted_labels
# Reorder columns to match your desired output: ['ID','prediction','Primary']
results_df = results_df[['ID', 'prediction', 'Primary']]

In [ ]:
# Reset index and concatenate with the probability DataFrame
results_df.reset_index(drop=True, inplace=True)
preds_proba_df.reset_index(drop=True, inplace=True)
final_results = pd.concat([results_df, preds_proba_df], axis=1)
# Optionally, rename the probability columns (here they are already set via target_labels)
final_results.columns = ['ID', 'prediction', 'Primary'] + target_labels

In [ ]:
final_results.to_csv('.../data/test_output/validation_res.csv', sep=',')